Modify the Decision Tree scratch code in our lecture such that:
- Modify the scratch code so it can accept an hyperparameter <code>max_depth</code>, in which it will continue create the tree until max_depth is reached.</li>
- Put everything into a class <code>DecisionTree</code>.  It should have at least two methods, <code>fit()</code>, and <code>predict()</code>
- Load the iris data and try with your class</li>

In [39]:
import numpy as np

class Node:
    def __init__(self, predicted_class):
        self.predicted_class = predicted_class
        self.feature_index = 0
        self.threshold = 0
        self.left = None
        self.right = None


class DecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth

    def fit(self, X, y):
        self.n_classes_ = len(set(y))
        self.n_features_ = X.shape[1]
        self.tree_ = self._grow_tree(X, y)

    def predict(self, X):
        return [self._predict(inputs) for inputs in X]

    def _best_split(self, X, y):
        m = y.size
        if m <= 1:
            return None, None
        num_parent = [np.sum(y == c) for c in range(self.n_classes_)]
        best_gini = 1.0 - sum((n / m) ** 2 for n in num_parent)
        best_idx, best_thr = None, None
        for idx in range(self.n_features_):
            thresholds, classes = zip(*sorted(zip(X[:, idx], y)))
            num_left = [0] * self.n_classes_
            num_right = num_parent.copy()
            for i in range(1, m):
                c = classes[i - 1]
                num_left[c] += 1
                num_right[c] -= 1
                gini_left = 1.0 - sum(
                    (num_left[x] / i) ** 2 for x in range(self.n_classes_)
                )
                gini_right = 1.0 - sum(
                    (num_right[x] / (m - i)) ** 2 for x in range(self.n_classes_)
                )
                gini = (i * gini_left + (m - i) * gini_right) / m
                if thresholds[i] == thresholds[i - 1]:
                    continue
                if gini < best_gini:
                    best_gini = gini
                    best_idx = idx
                    best_thr = (thresholds[i] + thresholds[i - 1]) / 2
        return best_idx, best_thr

    def _grow_tree(self, X, y, depth=0):
        num_samples_per_class = [np.sum(y == i) for i in range(self.n_classes_)]
        predicted_class = np.argmax(num_samples_per_class)
        node = Node(predicted_class=predicted_class)
        if depth < self.max_depth:
            idx, thr = self._best_split(X, y)
            if idx is not None:
                indices_left = X[:, idx] < thr
                X_left, y_left = X[indices_left], y[indices_left]
                X_right, y_right = X[~indices_left], y[~indices_left]
                node.feature_index = idx
                node.threshold = thr
                node.left = self._grow_tree(X_left, y_left, depth + 1)
                node.right = self._grow_tree(X_right, y_right, depth + 1)
                print(depth,idx, thr, node.left, node.right)
        return node

    def _predict(self, inputs):
        node = self.tree_
        while node.left:
            if inputs[node.feature_index] < node.threshold:
                node = node.left
            else:
                node = node.right
        return node.predicted_class


if __name__ == "__main__":
    import sys
    from sklearn.datasets import load_iris

    dataset = load_iris()
    X, y = dataset.data, dataset.target
    clf = DecisionTree(max_depth=10)
    clf.fit(X, y)
    print(clf.predict([[0, 0, 5, 1.5]]))

3 0 5.95 <__main__.Node object at 0x000001B5C7595910> <__main__.Node object at 0x000001B5C75953D0>
2 2 4.85 <__main__.Node object at 0x000001B5CAE8ED60> <__main__.Node object at 0x000001B5C76A2A60>
4 0 6.95 <__main__.Node object at 0x000001B5C77334C0> <__main__.Node object at 0x000001B5C7733790>
3 3 1.55 <__main__.Node object at 0x000001B5CAED3AC0> <__main__.Node object at 0x000001B5CA163700>
3 3 1.65 <__main__.Node object at 0x000001B5CAED37C0> <__main__.Node object at 0x000001B5CAED3FD0>
2 2 4.95 <__main__.Node object at 0x000001B5CAED3EE0> <__main__.Node object at 0x000001B5CA163970>
1 3 1.75 <__main__.Node object at 0x000001B5CA163D90> <__main__.Node object at 0x000001B5CAE8ED30>
0 2 2.45 <__main__.Node object at 0x000001B5CAEDB610> <__main__.Node object at 0x000001B5CAEE0610>
[2]


In [40]:
Xtrain = np.array([[2, 5],[3, 5],[10, 5],[19, 5]])
ytrain = np.array([0, 0, 1, 1])
Xtest = np.array(([[4, 6],[6, 9],[9, 2],[12, 8]]))
ytest = np.array([0, 0, 1, 1])

model = DecisionTree(max_depth=3)

model.fit(Xtrain, ytrain)
pred = [model.predict([x]) for x in Xtest]


print("Pred: ", np.array(pred))
print("ytest: ", ytest)

0 0 6.5 <__main__.Node object at 0x000001B5CAEA3BB0> <__main__.Node object at 0x000001B5CAEA3130>
Pred:  [[0]
 [0]
 [1]
 [1]]
ytest:  [0 0 1 1]


In [38]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

dataset = load_iris()
X, y = dataset.data, dataset.target
print(X.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                test_size=0.3, shuffle=True, random_state=42)

clf = DecisionTree(max_depth=3)
clf.fit(X_train, y_train)
pred = [clf.predict([x]) for x in X_test]
print((pred))
print((y_test))

print(clf.tree_.left)
print(clf.tree_.right)

(150, 4)
2 3 1.6 <__main__.Node object at 0x000001B5CAE8EE80> <__main__.Node object at 0x000001B5CAE8EFA0>
2 3 1.75 <__main__.Node object at 0x000001B5C7595C10> <__main__.Node object at 0x000001B5C7595DC0>
1 2 4.75 <__main__.Node object at 0x000001B5C76A2B80> <__main__.Node object at 0x000001B5CAEE0610>
0 2 2.45 <__main__.Node object at 0x000001B5CAE8ED90> <__main__.Node object at 0x000001B5C75A9310>
[[1], [0], [2], [1], [1], [0], [1], [2], [1], [1], [2], [0], [0], [0], [0], [2], [2], [1], [1], [2], [0], [2], [0], [2], [2], [2], [2], [2], [0], [0], [0], [0], [1], [0], [0], [2], [1], [0], [0], [0], [2], [2], [1], [0], [0]]
[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0 0 0 1 0 0 2 1
 0 0 0 2 1 1 0 0]
